## Setup

In [81]:
# File and text handling
import glob
import os
import re
import pandas as pd
from bs4 import BeautifulSoup
from near_regex import NEAR_regex
from tqdm import tqdm

# Gathering 2022 returns
from zipfile import ZipFile
from urllib.request import urlopen
from io import BytesIO

# Get filing dates from SEC EDGAR
from requests_html import HTMLSession
from time import sleep

In [82]:
# Paths and file handling
input_dir = 'inputs'
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

# Inputs
topic_path = input_dir + '/topic_list.csv'
sp500_path = input_dir + '/s&p500_2022.csv'
firm_10k_path = '10k_files/sec-edgar-filings'
firm_10k_clean_path = '10k_files/clean'

# Outputs
sentiment_save_path = output_dir + '/ticker_sentiments.csv'
returns_save_path = output_dir + '/ticker_returns.csv'
final_save_path = output_dir + '/analysis_sample.csv'

In [84]:
# Load S&P500 companies into a dataframe
try:
    sp500_orig = pd.read_csv(sp500_path)[['Symbol', 'Security', 'CIK', 'truth_path']]
except Exception as error:
    print('Please run the contents of download_text_files.ipynb before proceeding')
    print(repr(error))
    
sp500_orig

,Symbol,Security,CIK,truth_path
0,MMM,3M,66740,MMM
1,AOS,A. O. Smith,91142,AOS
2,ABT,Abbott,1800,ABT
3,ABBV,AbbVie,1551152,ABBV
4,ACN,Accenture,1467373,ACN
...,...,...,...,...
498,YUM,Yum! Brands,1041061,YUM
499,ZBRA,Zebra Technologies,877212,ZBRA
500,ZBH,Zimmer Biomet,1136869,ZBH
501,ZION,Zions Bancorporation,109380,0000109380


## Load 2022 returns

In [85]:
# Store the 2022 returns in a separate table
sp500_rets = sp500_orig.copy()

In [86]:
# Get the filing date for each 10-K
session = HTMLSession()

for i in tqdm(range(len(sp500_rets))):  # TODO:
    tic = sp500_rets['Symbol'].iloc[i]
    cik = sp500_rets['CIK'].iloc[i]
    truth_path = sp500_rets['truth_path'].iloc[i]
    
    if not os.path.exists(fr'{firm_10k_path}/{truth_path}/10-K/'):
        print(f'Error finding accession number for {tic}')
        print(fr'{firm_10k_path}/{truth_path}/10-K/')
        continue
    accession = os.listdir(fr'{firm_10k_path}/{truth_path}/10-K/')[0]
    
    url = f'https://www.sec.gov/Archives/edgar/data/{cik}/{accession}-index.html'
    r = session.get(url)
    sp500_rets.loc[i, 'filing_date'] = r.html.find('.info', first=True).text

sp500_rets

 40%|████████████████████████████████▏                                                | 200/503 [00:10<00:15, 19.44it/s]

Error finding accession number for FRC
10k_files/sec-edgar-filings/0001132979/10-K/


 43%|██████████████████████████████████▌                                              | 215/503 [00:11<00:14, 20.02it/s]

Error finding accession number for GEHC
10k_files/sec-edgar-filings/0001932393/10-K/


 84%|███████████████████████████████████████████████████████████████████▊             | 421/503 [00:22<00:03, 22.10it/s]

Error finding accession number for SBNY
10k_files/sec-edgar-filings/0001288784/10-K/


100%|█████████████████████████████████████████████████████████████████████████████████| 503/503 [00:26<00:00, 18.91it/s]


,Symbol,Security,CIK,truth_path,filing_date
0,MMM,3M,66740,MMM,2022-02-09
1,AOS,A. O. Smith,91142,AOS,2022-02-11
2,ABT,Abbott,1800,ABT,2022-02-18
3,ABBV,AbbVie,1551152,ABBV,2022-02-18
4,ACN,Accenture,1467373,ACN,2022-10-12
...,...,...,...,...,...
498,YUM,Yum! Brands,1041061,YUM,2022-02-28
499,ZBRA,Zebra Technologies,877212,ZBRA,2022-02-10
500,ZBH,Zimmer Biomet,1136869,ZBH,2022-02-25
501,ZION,Zions Bancorporation,109380,0000109380,2022-02-25


In [87]:
# Download 2022 CSRP returns
url = "https://github.com/LeDataSciFi/data/raw/main/Stock%20Returns%20(CRSP)/crsp_2022_only.zip"
with urlopen(url) as request:
    data = BytesIO(request.read())

with ZipFile(data) as archive:
    with archive.open(archive.namelist()[0]) as stata:
        stock_rets = pd.read_stata(stata)

stock_rets

,ticker,date,ret
0,JJSF,2021-12-01,-0.011276
1,JJSF,2021-12-02,0.030954
2,JJSF,2021-12-03,0.000287
3,JJSF,2021-12-06,0.014362
4,JJSF,2021-12-07,0.012459
...,...,...,...
2594044,TSLA,2022-12-23,-0.017551
2594045,TSLA,2022-12-27,-0.114089
2594046,TSLA,2022-12-28,0.033089
2594047,TSLA,2022-12-29,0.080827


In [88]:
# Based on filing date, add return from t to t+2 and from t+3 to t+10
combined_rets = sp500_rets.merge(
        stock_rets.rename(columns={'ticker':'Symbol'}),
        on='Symbol',
        how='left',
        validate='1:m') # TODO: include more?
combined_rets = combined_rets.query('filing_date <= date').groupby('Symbol').head(11)
combined_rets['agg_ret'] = 1 + combined_rets['ret']

combined_rets['ret_t-t2'] = combined_rets.groupby('Symbol') \
        .head(3)['agg_ret'] \
        .cumprod() - 1
combined_rets['ret_t3-t10'] = combined_rets.groupby('Symbol') \
        .tail(8)['agg_ret'] \
        .cumprod() - 1
final_rets = combined_rets.groupby('Symbol') \
        .head(3).groupby('Symbol') \
        .tail(1)[['Symbol', 'Security', 'CIK', 'filing_date', 'ret_t-t2']]
final_rets = final_rets.merge(
        combined_rets.groupby('Symbol') \
                .tail(1)[['Symbol', 'Security', 'CIK', 'ret_t3-t10']],
        on=['Symbol', 'Security', 'CIK'],
        validate='1:1',
        how='left') # TODO: include more?

final_rets.to_csv(returns_save_path, index=False)
final_rets

,Symbol,Security,CIK,filing_date,ret_t-t2,ret_t3-t10
0,MMM,3M,66740,2022-02-09,-0.017671,-0.090256
1,AOS,A. O. Smith,91142,2022-02-11,-0.014543,-0.138970
2,ABT,Abbott,1800,2022-02-18,-0.041758,-0.127147
3,ABBV,AbbVie,1551152,2022-02-18,-0.029927,-0.111148
4,ACN,Accenture,1467373,2022-10-12,-0.026583,-0.015658
...,...,...,...,...,...,...
492,YUM,Yum! Brands,1041061,2022-02-28,1.334293,-0.993641
493,ZBRA,Zebra Technologies,877212,2022-02-10,0.967095,-0.993766
494,ZBH,Zimmer Biomet,1136869,2022-02-25,1.018280,-0.994033
495,ZION,Zions Bancorporation,109380,2022-02-25,0.957101,-0.994100


## Clean 10-Ks

In [107]:
os.makedirs(firm_10k_clean_path, exist_ok=True)
sp500_sents = sp500_orig.copy()

for i in tqdm(range(len(sp500_sents))):
    tic = sp500_sents['Symbol'].iloc[i]
    truth_path = sp500_sents['truth_path'].iloc[i]    # TODO: explain

    # Check existence of path
    if not os.path.exists(fr'{firm_10k_path}/{truth_path}'):
        print(f'Cannot find 10-K for ticker {tic}')
        continue
    
    # Create clean path
    os.makedirs(fr'{firm_10k_clean_path}/{truth_path}', exist_ok=True)

    for path in glob.glob(fr'{firm_10k_path}/{truth_path}/*/*/*.html'):
        # Open and clean the 10-K
        with open(path, 'rb') as report_file:
            html = report_file.read()
        soup = BeautifulSoup(html, 'lxml-xml')
        for div in soup.find_all("div", {'style': 'display:none'}):
            div.decompose()                       # remove hidden divs,
        lower = soup.get_text().lower()           # uppercase,
        no_punc = re.sub(r'\W', ' ', lower)       # non-alpha-numeric,
        cleaned = re.sub(r'\s+', ' ', no_punc)    # single-space
        
        # Persist changes to the clean directory
        result_path = fr'{firm_10k_clean_path}/{truth_path}/10-K.txt' 
        with open(result_path, 'wb') as result_file:
            result_file.write(cleaned.encode('utf-8'))

  1%|▊                                                                                  | 5/503 [00:06<10:41,  1.29s/it]


KeyboardInterrupt: 

## Load sentiment dictionaries

TODO: justify that positive values as of 2021 are relevant.

In [95]:
# ML Dictionaries
with open('inputs/ML_negative_unigram.txt', 'r') as file:
    BHR_negative = [line.strip() for line in file]
with open('inputs/ML_positive_unigram.txt', 'r') as file:
    BHR_positive = [line.strip() for line in file]

In [96]:
# LM Dictionaries
LM = pd.read_csv('inputs/LM_MasterDictionary_1993-2021.csv')
LM_negative = LM.query('Negative > 0')['Word'].to_list()
LM_positive = LM.query('Positive > 0')['Word'].to_list()

## Load each firm and add sentiment variables

TODO: load and clean

For each firm, 

1. [x] load the corresponding 10-K. Clean the text.

1. [x] Create the sentiment measurements, and save those new measurements to the correct row and column in the dataframe.

1. [x] Bonus: Save the total length of the document (# of words)

1. [x] Bonus: Save the # of unique words (similar to total length)

1. [x] Calculate returns from t to t+2 inclusive

1. [x] Calculate returns from t+3 to t+10 inclusive

1. [ ] Download 2021 accounting data (2021 ccm_cleaned.dta) from the data repo (possibly useful in analysis) and add to dataset

In [97]:
# Gather sentiments into regex
BHR_negative_regex = '(' + '|'.join(BHR_negative).lower() + ')'
BHR_positive_regex = '(' + '|'.join(BHR_positive).lower() + ')'
LM_negative_regex = '(' + '|'.join(LM_negative).lower() + ')'
LM_positive_regex = '(' + '|'.join(LM_positive).lower() + ')'

In [98]:
# Topic regex
def NEAR_regex_helper(topic_list, valence_list, max_words_between=5):
    # print(topic_list.apply(NEAR_regex, args=(5,True)))
    # TODO: partial match for each word
    topic_regex = '(' + '|'.join(topic_list).lower() + ')'
    valence_regex = '(' + '|'.join(valence_list).lower() + ')'
    return NEAR_regex([topic_regex, valence_regex], max_words_between=max_words_between)

# Read in file
if not os.path.exists(topic_path):
    print(f'Cannot find path {topic_path} to topic list')
else:
    topic_df = pd.read_csv(topic_path)
    
    # ESG
    esg_topic = topic_df['term'].loc[topic_df['type'] == 'esg_topic']
    esg_negative = topic_df['term'].loc[topic_df['type'] == 'esg_negative']
    esg_positive = topic_df['term'].loc[topic_df['type'] == 'esg_positive']

    # Ecommerce
    ecom_topic = topic_df['term'].loc[topic_df['type'] == 'ecom_topic']
    ecom_negative = topic_df['term'].loc[topic_df['type'] == 'ecom_negative']
    ecom_positive = topic_df['term'].loc[topic_df['type'] == 'ecom_positive']

    # Bio
    bio_topic = topic_df['term'].loc[topic_df['type'] == 'bio_topic']
    bio_negative = topic_df['term'].loc[topic_df['type'] == 'bio_negative']
    bio_positive = topic_df['term'].loc[topic_df['type'] == 'bio_positive']

    # Generate regex
    esg_negative_regex = NEAR_regex_helper(esg_topic, esg_negative)
    esg_positive_regex = NEAR_regex_helper(esg_topic, esg_positive)
    ecom_negative_regex = NEAR_regex_helper(ecom_topic, ecom_negative)
    ecom_positive_regex = NEAR_regex_helper(ecom_topic, ecom_positive)
    bio_negative_regex = NEAR_regex_helper(bio_topic, bio_negative)
    bio_positive_regex = NEAR_regex_helper(bio_topic, bio_positive)

In [99]:
# # TODO: remove duplicated code here
# if not os.path.exists(sp500_path):
#     url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
#     pd.read_html(url)[0].to_csv(sp500_path, index=False)  # [1] shows updates

# sp500 = pd.read_csv(sp500_path)[['Symbol', 'Security', 'CIK']]
# sp500_sents = sp500_orig.copy()

In [100]:
# Simplify adding a value to a given row
def add_sentiment(df, i, sentiment_name, search, text, word_count):
    df.loc[i, sentiment_name] = len(re.findall(search, text)) / word_count

In [103]:
sp500_sents

,Symbol,Security,CIK,word_count,bhr_negative,bhr_positive,lm_negative,lm_positive,esg_negative,esg_positive,ecom_negative,ecom_positive,bio_negative,bio_positive,unique_word_count
0,MMM,3M,66740,76432.0,0.044314,0.041750,0.043097,0.015125,0.000000,0.000013,0.000000,0.000000,0.000052,0.000118,6384.0
1,AOS,A. O. Smith,91142,33810.0,0.032919,0.038391,0.034102,0.013162,0.000030,0.000000,0.000000,0.000000,0.000000,0.000089,3801.0
2,ABT,Abbott,1800,52061.0,0.039492,0.039761,0.036246,0.010584,0.000019,0.000000,0.000000,0.000038,0.000115,0.000365,5045.0
3,ABBV,AbbVie,1551152,61560.0,0.035185,0.034958,0.035819,0.014766,0.000016,0.000016,0.000000,0.000016,0.000065,0.000276,5886.0
4,ACN,Accenture,1467373,51953.0,0.034127,0.046754,0.036437,0.018440,0.000000,0.000058,0.000019,0.000038,0.000000,0.000058,5169.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,1041061,84234.0,0.032873,0.041052,0.034630,0.013771,0.000059,0.000000,0.000024,0.000059,0.000012,0.000024,4945.0
499,ZBRA,Zebra Technologies,877212,46978.0,0.038273,0.044680,0.033846,0.019775,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4750.0
500,ZBH,Zimmer Biomet,1136869,64913.0,0.038544,0.035232,0.039176,0.013233,0.000015,0.000031,0.000000,0.000000,0.000031,0.000169,4577.0
501,ZION,Zions Bancorporation,109380,61897.0,0.029307,0.028208,0.028951,0.008756,0.000016,0.000032,0.000000,0.000000,0.000000,0.000000,3818.0


In [109]:
# TODO: uncomment if not

# Load and get the sentiment of clean 10-Ks
temp_path = output_dir + '/ticker_sentiments_temp.csv'  # TODO: add *temp* to .gitignore

# if not os.path.exists(sentiment_save_path):
for i in tqdm(range(len(sp500_sents))):
    tic = sp500_sents['Symbol'].iloc[i]
    truth_path = sp500_sents['truth_path'].iloc[i]    # TODO: explain

    # Check existence of path
    if not os.path.exists(fr'{firm_10k_path}/{truth_path}'): # TODO: 10k_path to 10k_clean_path
        print(f'Cannot find 10-K for ticker {tic}')
        continue

    for path in glob.glob(fr'{firm_10k_clean_path}/{truth_path}/*.txt'):
        with open(path, 'rb') as report_file:
            cleaned = str(report_file.read())

        # Add word count and unique word count
        word_list = re.findall(r'\w+', cleaned)
        sp500_sents.loc[i, 'unique_word_count'] = len(set(word_list))
        word_count = len(word_list)
        sp500_sents.loc[i, 'word_count'] = word_count

        # Gather valence variables
        add_sentiment(sp500_sents, i, 'bhr_negative', BHR_negative_regex, cleaned, word_count)
        add_sentiment(sp500_sents, i, 'bhr_positive', BHR_positive_regex, cleaned, word_count)
        add_sentiment(sp500_sents, i, 'lm_negative', LM_negative_regex, cleaned, word_count)
        add_sentiment(sp500_sents, i, 'lm_positive', LM_positive_regex, cleaned, word_count)

        # Gather topic valence variables
        add_sentiment(sp500_sents, i, 'esg_negative', esg_negative_regex, cleaned, word_count)
        add_sentiment(sp500_sents, i, 'esg_positive', esg_positive_regex, cleaned, word_count)
        add_sentiment(sp500_sents, i, 'ecom_negative', ecom_negative_regex, cleaned, word_count)
        add_sentiment(sp500_sents, i, 'ecom_positive', ecom_positive_regex, cleaned, word_count)
        add_sentiment(sp500_sents, i, 'bio_negative', bio_negative_regex, cleaned, word_count)
        add_sentiment(sp500_sents, i, 'bio_positive', bio_positive_regex, cleaned, word_count)

        # Save intermittently
        if i % 25 == 0:
            sp500_sents.to_csv(temp_path, index=False)

sp500_sents.to_csv(sentiment_save_path, index=False)
# else:
#     # Load existing sentiments
#     sp500_sents = pd.read_csv(sentiment_save_path)
#     pass
sp500_sents.describe()

 39%|███████████████████████████████▋                                                 | 197/503 [14:42<27:59,  5.49s/it]

Cannot find 10-K for ticker FRC


 42%|██████████████████████████████████▎                                              | 213/503 [15:51<19:20,  4.00s/it]

Cannot find 10-K for ticker GEHC


 83%|███████████████████████████████████████████████████████████████████▏             | 417/503 [30:30<06:12,  4.33s/it]

Cannot find 10-K for ticker SBNY


100%|█████████████████████████████████████████████████████████████████████████████████| 503/503 [36:31<00:00,  4.36s/it]


,CIK,unique_word_count,word_count,bhr_negative,bhr_positive,lm_negative,lm_positive,esg_negative,esg_positive,ecom_negative,ecom_positive,bio_negative,bio_positive
count,5.030000e+02,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,7.940509e+05,5425.424000,69504.586000,0.038030,0.039918,0.036081,0.014795,0.000033,0.000022,0.000003,0.000017,0.000019,0.000097
std,5.545629e+05,1177.831997,28943.930805,0.004226,0.005064,0.004679,0.002147,0.000060,0.000032,0.000008,0.000029,0.000054,0.000190
min,1.800000e+03,460.000000,1575.000000,0.015659,0.010931,0.023693,0.004394,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.761050e+04,4767.250000,51384.750000,0.035457,0.036979,0.033086,0.013601,0.000000,0.000000,0.000000,0.000000,0.000000,0.000017
50%,8.848870e+05,5324.500000,64916.500000,0.038217,0.039824,0.035853,0.014822,0.000017,0.000015,0.000000,0.000000,0.000000,0.000047
75%,1.137954e+06,5893.500000,80526.000000,0.040611,0.043145,0.038983,0.016067,0.000042,0.000032,0.000000,0.000023,0.000017,0.000107
max,1.932393e+06,10469.000000,271760.000000,0.051376,0.056662,0.055229,0.021395,0.000945,0.000458,0.000077,0.000278,0.000505,0.002569


## Get 2021 CCM Data

TODO: load and clean


In [113]:
# TODO: fix download
# pd.set_option('display.max_columns', None)
crsp_2021_url = 'https://github.com/LeDataSciFi/data/raw/main/Firm%20Year%20Datasets%20(Compustat)/2021_ccm_cleaned.dta'
crsp_2021 = pd.read_stata(crsp_2021_url)
sp500_sents.merge(crsp_2021.rename(columns={'tic':'Symbol'}),
        on='Symbol',
        how='left',
        validate='1:1') # TODO: include more?
# print(crsp_2021.head())
# crsp_2021.describe()

,Symbol,Security,CIK,truth_path,unique_word_count,word_count,bhr_negative,bhr_positive,lm_negative,lm_positive,esg_negative,esg_positive,ecom_negative,ecom_positive,bio_negative,bio_positive,gvkey,fyear,lpermno,lpermco,datadate,sic,sic3,td,long_debt_dum,me,l_a,l_sale,capx_a,div_d,age,atr,smalltaxlosscarry,largetaxlosscarry,gdpdef,l_reala,l_reallongdebt,kz_index,ww_index,hp_index,ww_unconstrain,ww_constrained,kz_unconstrain,kz_constrained,hp_unconstrain,hp_constrained,tnic3tsimm,tnic3hhi,prodmktfluid,delaycon,equitydelaycon,debtdelaycon,privdelaycon,at_raw,raw_Inv,raw_Ch_Cash,raw_Div,raw_Ch_Debt,raw_Ch_Eqty,raw_Ch_WC,raw_CF,l_emp,l_ppent,l_laborratio,Inv,Ch_Cash,Div,Ch_Debt,Ch_Eqty,Ch_WC,CF,td_a,td_mv,mb,prof_a,ppe_a,cash_a,xrd_a,dltt_a,invopps_FG09,sales_g,dv_a,short_debt
0,MMM,3M,66740,MMM,6384.0,76433.0,0.044313,0.041749,0.043097,0.015124,0.000000,0.000013,0.000000,0.000000,0.000052,0.000118,7435.0,2021.0,22592.0,21205.0,2021-12-31,2670.0,267.0,18317.000000,1.0,101576.828125,10.759434,10.473195,0.034054,1.0,2.0,0.178324,NaN,NaN,121.708000,5.957809,4.931176,-11.940228,-0.534082,-2.944599,1.0,0.0,1.0,0.0,1.0,0.0,1.000000,1.000000,1.286998,NaN,NaN,NaN,NaN,47072.000000,0.027978,-0.001487,0.072655,-0.024324,-0.033141,0.016379,0.156611,4.564348,9.238733,4.684759,0.027978,-0.001487,0.072655,-0.024324,-0.033141,0.016379,0.156611,0.389127,0.152777,2.838265,0.197931,0.218538,0.101228,0.042361,0.355625,2.564301,0.098527,0.072655,0.086095
1,AOS,A. O. Smith,91142,AOS,3801.0,33811.0,0.032918,0.038390,0.034101,0.013161,0.000030,0.000000,0.000000,0.000000,0.000000,0.000089,9771.0,2021.0,65402.0,21625.0,2021-12-31,3630.0,363.0,230.699997,1.0,13534.509766,8.153177,8.171571,0.021615,1.0,2.0,0.221387,NaN,NaN,121.708000,3.351552,1.009241,NaN,-0.434246,-2.067079,1.0,0.0,0.0,NaN,1.0,0.0,1.000000,1.000000,1.650701,NaN,NaN,NaN,NaN,3474.399902,0.100708,-0.037359,0.048958,0.024033,-0.096247,-0.018478,0.184521,2.687847,6.461781,3.842822,0.100708,-0.037359,0.048958,0.024033,-0.096247,-0.018478,0.184521,0.066400,0.016760,4.368153,0.197847,0.183974,0.181729,0.027113,0.061075,NaN,0.222291,0.048958,0.080191
2,ABT,Abbott,1800,ABT,5047.0,52066.0,0.039488,0.039757,0.036242,0.010583,0.000019,0.000000,0.000000,0.000038,0.000115,0.000365,1078.0,2021.0,20482.0,20017.0,2021-12-31,3845.0,384.0,19251.000000,1.0,248276.906250,11.227854,10.670698,0.025068,1.0,2.0,0.138838,NaN,NaN,121.708000,6.426229,5.017051,-12.777783,-0.525380,-3.040385,1.0,0.0,1.0,0.0,1.0,0.0,1.136600,0.300567,3.067555,NaN,NaN,NaN,NaN,75196.000000,0.026704,0.039377,0.042582,-0.000585,-0.027182,0.012966,0.136430,4.736198,9.221577,4.494090,0.026704,0.039377,0.042582,-0.000585,-0.027182,0.012966,0.136430,0.256011,0.071959,3.825614,0.166285,0.134475,0.136297,0.036465,0.242726,3.559664,0.244654,0.042582,0.051893
3,ABBV,AbbVie,1551152,ABBV,5890.0,61568.0,0.035181,0.034953,0.035814,0.014764,0.000016,0.000016,0.000000,0.000016,0.000065,0.000276,16101.0,2021.0,13721.0,54287.0,2021-12-31,2836.0,283.0,77575.000000,1.0,239432.968750,11.894979,10.936619,0.005371,1.0,2.0,0.110923,0.0,1.0,121.708000,7.093354,6.280882,-20.090805,0.833240,-3.144228,0.0,1.0,1.0,0.0,1.0,0.0,32.004902,0.073033,5.961626,NaN,NaN,NaN,NaN,146529.000000,0.015997,0.008851,0.063203,-0.057422,-0.004709,0.013622,0.150182,3.931826,8.678121,4.765928,0.015997,0.008851,0.063203,-0.057422,-0.004709,0.013622,0.150182,0.529417,0.244710,2.528878,0.194432,0.040074,0.067086,0.054911,0.442929,2.144449,0.227438,0.063203,0.163364
4,ACN,Accenture,1467373,ACN,5171.0,51961.0,0.034122,0.046747,0.036431,0.018437,0.000000,0.000058,0.000019,0.000038,0.000000,0.000058,143357.0,2021.0,89071.0,41871.0,2021-08-31,8742.0,874.0,3506.634033,1.0,212734.859375,10.673037,10.830390,0.013436,1.0,2.0,0.230622,1.0,0.0,119.711998,5.887948,3.177014,-19.948496,-0.450053,-2.928696,1.0,0.0,1.0,0.0,1.0,0.0,1.186600,0.201419,3.732553,NaN,NaN,NaN,NaN,43175.843750,0.099819,-0.005724,0.051790,-0.000181,-0.061084,0.000296,0.207149,5.247024,8.481073,2.044716,0.099819,-0.005724,0.051790,-0.

In [35]:
sp500_orig.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


## Merge Sentiment Scores, Returns, and 2021 CCM Data

In [68]:
sp500_final = final_rets.merge(sp500_sents, on=['Symbol', 'Security', 'CIK'], validate='1:1', how='outer')
sp500_final.to_csv(final_save_path, index=False)
sp500_final

,Symbol,Security,CIK,filing_date,ret_t-t2,ret_t3-t10,word_count,bhr_negative,bhr_positive,lm_negative,lm_positive,esg_negative,esg_positive,ecom_negative,ecom_positive,bio_negative,bio_positive,unique_word_count
0,MMM,3M,66740,2022-02-09,-0.017671,-0.090256,76432.0,0.044314,0.041750,0.043097,0.015125,0.000000,0.000013,0.000000,0.000000,0.000052,0.000118,6384.0
1,AOS,A. O. Smith,91142,2022-02-11,-0.014543,-0.138970,33810.0,0.032919,0.038391,0.034102,0.013162,0.000030,0.000000,0.000000,0.000000,0.000000,0.000089,3801.0
2,ABT,Abbott,1800,2022-02-18,-0.041758,-0.127147,52061.0,0.039492,0.039761,0.036246,0.010584,0.000019,0.000000,0.000000,0.000038,0.000115,0.000365,5045.0
3,ABBV,AbbVie,1551152,2022-02-18,-0.029927,-0.111148,61560.0,0.035185,0.034958,0.035819,0.014766,0.000016,0.000016,0.000000,0.000016,0.000065,0.000276,5886.0
4,ACN,Accenture,1467373,2022-10-12,-0.026583,-0.015658,51953.0,0.034127,0.046754,0.036437,0.018440,0.000000,0.000058,0.000019,0.000038,0.000000,0.000058,5169.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,FRC,First Republic Bank,1132979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499,GEHC,GE HealthCare,1932393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500,SBNY,Signature Bank,1288784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501,WBD,Warner Bros. Discovery,1437107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## TODO: OLD

In [17]:
# # Topic regex
# def NEAR_regex_helper(topic_list, valence_list, max_words_between=5):
#     topic_regex = '(' + '|'.join(topic_list).lower() + ')'
#     valence_regex = '(' + '|'.join(valence_list).lower() + ')'
#     return NEAR_regex([topic_regex, valence_regex], max_words_between=max_words_between)

# # Socially responsible investing
# esg_topics = ['esg', 'sustainable', 'sustainability', 'impact invest',
#               'clean energy', 'gender', 'diversity', 'inclusion',
#               'microfinance', 'ethical', 'cdfi']
# esg_negative = ['limited', 'limit', 'underdeveloped', 'underdevelop',
#                 'bureaucratic', 'slow', 'insufficient']
# esg_positive = ['ethical', 'sustainable', 'profitable', 'profit',
#                 'innovative', 'innovation', 'transformative', 'transform']
# esg_negative_regex = NEAR_regex_helper(esg_topics, esg_negative)
# esg_positive_regex = NEAR_regex_helper(esg_topics, esg_positive)

# # Ecommerce
# ecom_topics = ['online', 'digital payment', 'logistics', 'delivery', 
#                'mobile commerce', 'social commerce', 'dropship',
#                'drop ship', 'social media']
# ecom_negative = ['risky', 'unsustainable',  'unsustained', 'monopoly',
#                  'monopolistic', 'unethical']
# ecom_positive = ['convenienent', 'convenienence', 'accessible', 'access',
#                  'innovative', 'innovation', 'profitable', 'profit',
#                  'efficient']
# ecom_negative_regex = NEAR_regex_helper(ecom_topics, ecom_negative)
# ecom_positive_regex = NEAR_regex_helper(ecom_topics, ecom_positive)

# # Biotech and healthcare
# bio_topics = ['gene', 'biopharm', 'telemedic', 'personalized medic',
#               'medical device', 'vaccine', 'precision medic', 'organ',
#               'regenerative medic', 'prosthetic', 'clinic', 'fda',
#               'health']
# bio_negative = ['risky', 'expensive', 'slow', 'controversial', 'unethical']
# bio_positive = ['new', 'safe', 'innovative', 'innovation',
#                 'transformative', 'transform', 'life', 'lives']
# bio_negative_regex = NEAR_regex_helper(bio_topics, bio_negative)
# bio_positive_regex = NEAR_regex_helper(bio_topics, bio_positive)

In [18]:
# esg_negative_regex
# # esg_positive_regex
# # ecom_negative_regex
# # ecom_positive_regex
# # bio_negative_regex
# # bio_positive_regex